*Eduardus Tjitrahardja | @edutjie | 2022*

# Final Project - Forest Fire Estimation

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('seaborn')

## Loading Dataset
-  **Attribute Information:**
   - **X** - x-axis spatial coordinate within the Montesinho park map: 1 to 9
   - **Y** - y-axis spatial coordinate within the Montesinho park map: 2 to 9
   - **month** - month of the year: 'jan' to 'dec'
   - **day** - day of the week: 'mon' to 'sun'
   - **FFMC** - FFMC index from the FWI system: 18.7 to 96.20
   - **DMC** - DMC index from the FWI system: 1.1 to 291.3
   - **DC** - DC index from the FWI system: 7.9 to 860.6
   - **ISI** - ISI index from the FWI system: 0.0 to 56.10
   - **temp** - temperature in Celsius degrees: 2.2 to 33.30
   - **RH** - relative humidity in %: 15.0 to 100
   - **wind** - wind speed in km/h: 0.40 to 9.40
   - **rain** - outside rain in mm/m2 : 0.0 to 6.4
   - **area** - the burned area of the forest (in ha): 0.00 to 1090.84
   (this output variable is very skewed towards 0.0, thus it may make
   sense to model with the logarithm transform).

In [2]:
df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv")
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [3]:
print("DataFrame shape:", df.shape)
df.info()

DataFrame shape: (517, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       517 non-null    int64  
 1   Y       517 non-null    int64  
 2   month   517 non-null    object 
 3   day     517 non-null    object 
 4   FFMC    517 non-null    float64
 5   DMC     517 non-null    float64
 6   DC      517 non-null    float64
 7   ISI     517 non-null    float64
 8   temp    517 non-null    float64
 9   RH      517 non-null    int64  
 10  wind    517 non-null    float64
 11  rain    517 non-null    float64
 12  area    517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB
